In [8]:
from torch.optim.lr_scheduler import LRScheduler
import sys
from pprint import PrettyPrinter
pp = PrettyPrinter(indent=2)
from functools import partial

opt = getattr(sys.modules['torch.optim.lr_scheduler'], 'CosineAnnealingLR')
# print(opt)
kwarg = {'T_max': 10, 'eta_min': 1e-4}
lrs = partial(opt, **kwarg)
print(lrs)
# pp.pprint(list(sys.modules.keys()))

functools.partial(<class 'torch.optim.lr_scheduler.CosineAnnealingLR'>, T_max=10, eta_min=0.0001)


In [7]:
import numpy as np

x = [1, 2, 3]
y = [1.000001, 2.0000001, 3]
z = np.allclose(x, y, atol=1e-4)
print(z)
print(all(t > 1 for t in y))

True
True


In [19]:
import torch

a = torch.zeros(2, 1)
a[0] = 0.123
a[1] = 0.142
print(a)
print(a.sum())
print(a.sum().item())

tensor([[0.1230],
        [0.1420]])
tensor(0.2650)
0.26500001549720764


## Testing Dataloaders

In [9]:
import torch
import torch.nn as nn
from omegaconf import OmegaConf
import pickle

In [26]:
ks = (3, 3, 5)
nc = (10, 16, 32)
for i, (a, b) in enumerate(zip(nc, ks)):
    print(i, a, b)

0 10 3
1 16 3
2 32 5


In [30]:
ks = [1, 2, 34]
ks.insert(1, -12)
ks

[1, -12, 2, 34]

In [13]:
x = torch.rand(1, 3)
print(x)

y = torch.unsqueeze(x, 0).expand(4, -1, -1)
print(y)

tensor([[0.3484, 0.7877, 0.9607]])
tensor([[[0.3484, 0.7877, 0.9607]],

        [[0.3484, 0.7877, 0.9607]],

        [[0.3484, 0.7877, 0.9607]],

        [[0.3484, 0.7877, 0.9607]]])


In [6]:
x = torch.rand(4, 10)
y = torch.unsqueeze(x, 1).expand(32)
print(x.shape)
print(y.shape)

RuntimeError: expand(torch.FloatTensor{[4, 1, 10]}, size=[32]): the number of sizes provided (1) must be greater or equal to the number of dimensions in the tensor (3)

In [8]:
import pandas as pd

with open('../data/train100_mod.pkl', 'rb') as f:
    df = pickle.load(f)

df.info()
display(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28743 entries, 0 to 28742
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           28743 non-null  int64 
 1   video_id             28743 non-null  object
 2   root_dir             28743 non-null  object
 3   narration_id         28743 non-null  object
 4   narration            28743 non-null  object
 5   narration_timestamp  28743 non-null  object
 6   start_frame          28743 non-null  int64 
 7   end_frame            28743 non-null  int64 
 8   verb_class           28743 non-null  int64 
 9   noun_class           28743 non-null  int64 
dtypes: int64(5), object(5)
memory usage: 2.2+ MB


,Unnamed: 0,video_id,root_dir,narration_id,narration,narration_timestamp,start_frame,end_frame,verb_class,noun_class
0,0,P01_103,../2g1n6qdydwa9u22shpxqzp0t8m/P01,P01_103_0,close door,00:00:02.103,6,381,4,3
1,1,P01_103,../2g1n6qdydwa9u22shpxqzp0t8m/P01,P01_103_1,pick up plate,00:00:07.615,281,493,0,2
2,2,P01_103,../2g1n6qdydwa9u22shpxqzp0t8m/P01,P01_103_2,open cupboard,00:00:09.850,393,531,3,3
3,3,P01_103,../2g1n6qdydwa9u22shpxqzp0t8m/P01,P01_103_3,put plate into cupboard,00:00:10.600,431,593,5,2
4,4,P01_103,../2g1n6qdydwa9u22shpxqzp0t8m/P01,P01_103_4,close cupboard,00:00:11.850,493,668,4,3


In [21]:
df.groupby('video_id')['end_frame'].max().reset_index()

,video_id,end_frame
0,P01_103,8906
1,P01_105,106023
2,P01_106,36174
3,P01_107,5977
4,P01_108,7000
...,...,...
144,P35_108,150759
145,P35_109,56479
146,P37_101,84078
147,P37_102,34136


In [20]:
print(185406/50)
print(2**12)

3708.12
4096


In [31]:
def fn(bn, in_feat_dim, n_conv, k_sizes, dropout=0.1):
    if len(bn) > 0:
        layers = [
            [
                nn.Conv1d(ni, nc, kf, bias=False if i in bn else True), 
                nn.ReLU(), 
                nn.Dropout(dropout),
            ] 
            for i, (ni, nc, kf) in enumerate(zip(in_feat_dim, n_conv, k_sizes))
        ]
        for i in bn:
            layers[i].insert(2, nn.BatchNorm2d(n_conv[i]))

            
    else:
        layers = [
            [nn.Conv1d(ni, nc, kf, bias=True), nn.ReLU(), nn.Dropout(dropout),] 
            for ni, nc, kf in zip(in_feat_dim, n_conv, k_sizes)
        ]

    linear_layer = nn.Sequential(*layers)
    print(linear_layer)

In [32]:
fn([1, 3], [1, 8, 32, 16], [8, 32, 16, 10], [3, 5, 3, 3])

TypeError: list is not a Module subclass

In [2]:
config = OmegaConf.load('../configs/pilot_config.yaml')
# print(OmegaConf.to_yaml(config))

In [3]:
import sys
from pprint import PrettyPrinter

pp = PrettyPrinter()
pp.pprint(sys.path)

['/coc/scratch/bdas31/EPIC-Kitchens_100/HAAR/src',
 '/coc/pcba1/bdas31/Miniconda3/envs/haar-2/lib/python310.zip',
 '/coc/pcba1/bdas31/Miniconda3/envs/haar-2/lib/python3.10',
 '/coc/pcba1/bdas31/Miniconda3/envs/haar-2/lib/python3.10/lib-dynload',
 '',
 '/coc/pcba1/bdas31/Miniconda3/envs/haar-2/lib/python3.10/site-packages',
 '/coc/pcba1/bdas31/Miniconda3/envs/haar-2/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg',
 '/coc/pcba1/bdas31/Miniconda3/envs/haar-2/lib/python3.10/site-packages/huggingface_hub-0.16.4-py3.8.egg',
 '/coc/pcba1/bdas31/Miniconda3/envs/haar-2/lib/python3.10/site-packages/sacremoses-0.0.43-py3.8.egg']


In [12]:
sys.path.append('/coc/scratch/bdas31/EPIC-Kitchens_100/HAAR/src/datasets')
pp.pprint(sys.path)

['/coc/scratch/bdas31/EPIC-Kitchens_100/HAAR/src/scratch',
 '/coc/pcba1/bdas31/Miniconda3/envs/haar-2/lib/python310.zip',
 '/coc/pcba1/bdas31/Miniconda3/envs/haar-2/lib/python3.10',
 '/coc/pcba1/bdas31/Miniconda3/envs/haar-2/lib/python3.10/lib-dynload',
 '',
 '/coc/pcba1/bdas31/Miniconda3/envs/haar-2/lib/python3.10/site-packages',
 '/coc/pcba1/bdas31/Miniconda3/envs/haar-2/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg',
 '/coc/pcba1/bdas31/Miniconda3/envs/haar-2/lib/python3.10/site-packages/huggingface_hub-0.16.4-py3.8.egg',
 '/coc/pcba1/bdas31/Miniconda3/envs/haar-2/lib/python3.10/site-packages/sacremoses-0.0.43-py3.8.egg',
 '/coc/scratch/bdas31/EPIC-Kitchens_100/HAAR/',
 '/coc/scratch/bdas31/EPIC-Kitchens_100/HAAR/src',
 '/coc/scratch/bdas31/EPIC-Kitchens_100/HAAR/src/datasets']


In [4]:
from systems.data_module import EpicActionRecognitionDataModule


data_module = EpicActionRecognitionDataModule(config)

In [5]:
train_loader = data_module.test_dataloader()
print(type(train_loader))

<class 'torch.utils.data.dataloader.DataLoader'>


In [16]:
# %load_ext autoreload
# %autoreload 2
import models
import importlib
# importlib.reload(models)
from models.models import EmbeddingModel

device = 'cuda:0'
x = torch.rand(4, 32, 3, 224, 224, device=device)
x = [x for _ in range(4)]
model = EmbeddingModel(config, 0.1, device)
y = model(x, permute_dims=False)
print(y.shape)

2023-09-25 20:22:10,521 [MainThread  ] [WARNI]  model.pretrained was set to 'None' but you also specified to load weights from {'rgb': '/coc/scratch/bdas31/EPIC-Kitchens_100/C1-Action-Recognition-TSN-TRN-TSM/pretrained_models/tsm_rgb.ckpt', 'flow': '/coc/scratch/bdas31/EPIC-Kitchens_100/C1-Action-Recognition-TSN-TRN-TSM/pretrained_models/tsm_flow.ckpt'}.The latter will take precedence.
2023-09-25 20:22:11,028 [MainThread  ] [WARNI]  model.pretrained was set to 'None' but you also specified to load weights from {'rgb': '/coc/scratch/bdas31/EPIC-Kitchens_100/C1-Action-Recognition-TSN-TRN-TSM/pretrained_models/tsm_rgb.ckpt', 'flow': '/coc/scratch/bdas31/EPIC-Kitchens_100/C1-Action-Recognition-TSN-TRN-TSM/pretrained_models/tsm_flow.ckpt'}.The latter will take precedence.


ValueError: too many values to unpack (expected 2)

In [22]:
from models.models import EmbeddingModel


model = EmbeddingModel(config, 0.1, 'cuda:0')
for x in train_loader:
    # x = model(x, permute_dims=False)
    # pp.pprint(metadata)
    print(x[0][0].shape)
    break

2023-09-25 20:11:31,402 [MainThread  ] [WARNI]  model.pretrained was set to 'None' but you also specified to load weights from {'rgb': '/coc/scratch/bdas31/EPIC-Kitchens_100/C1-Action-Recognition-TSN-TRN-TSM/pretrained_models/tsm_rgb.ckpt', 'flow': '/coc/scratch/bdas31/EPIC-Kitchens_100/C1-Action-Recognition-TSN-TRN-TSM/pretrained_models/tsm_flow.ckpt'}.The latter will take precedence.
2023-09-25 20:11:31,778 [MainThread  ] [WARNI]  model.pretrained was set to 'None' but you also specified to load weights from {'rgb': '/coc/scratch/bdas31/EPIC-Kitchens_100/C1-Action-Recognition-TSN-TRN-TSM/pretrained_models/tsm_rgb.ckpt', 'flow': '/coc/scratch/bdas31/EPIC-Kitchens_100/C1-Action-Recognition-TSN-TRN-TSM/pretrained_models/tsm_flow.ckpt'}.The latter will take precedence.
torch.Size([4, 32, 3, 224, 224])


In [15]:
y = torch.mean(x, dim=1)
print(y.shape)

torch.Size([4, 3, 224, 224])


In [2]:
import torch
import torch.nn as nn
from pathlib import Path
from transformers import Trainer, TrainingArguments
from omegaconf import OmegaConf, DictConfig
import re

import sys
sys.path.append(sys.path[0] + "/..")
from models.models import AttentionModel
from systems.data_module import EpicActionRecognitionDataModule
from systems.recognition_module import EpicActionRecognitionModule
from train import create_snapshot_paths

ModuleNotFoundError: No module named 'datasets.epic_dataset'

In [10]:
cfg = OmegaConf.load('../../configs/pilot_config.yaml')
# device = torch.device('cuda:0')
data_module = EpicActionRecognitionDataModule(cfg)
verb_path, noun_path = create_snapshot_paths(Path(cfg.model.save_path))
system = EpicActionRecognitionModule(cfg)
# system.load_models_to_device(verb=True)
# model = AttentionModel(system.verb_map)


In [3]:
%reload_ext autoreload
%autoreload 2
from models.attention import Transformer


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)

# x = torch.tensor([[1, 5, 6, 4, 3, 3, 5, 2, 0], [1, 1, 7, 3, 4, 5, 6, 7, 2]]).to(
#     device
# )
# trg = torch.tensor([[1, 7, 4, 3, 5, 3, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

# src_pad_idx = 0
# trg_pad_idx = 0
# src_vocab_size = 8
# trg_vocab_size = 8
# model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(
#     device
# )
# # out = model(x, trg[:, :-1])
# # print(out.shape)

In [4]:
from constants import NUM_VERBS, WORD_EMBEDDING_SIZE
from models.models import EmbeddingModel


class HaarModel(nn.Module):
    def __init__(self, verb_map, noun_map, dropout, device):
        super().__init__()
        self.verb_map = verb_map
        self.noun_map = noun_map

        self.device = device
        self.feature_model = EmbeddingModel(cfg, dropout, device)
        # self.transformer = nn.Transformer(
        #     d_model=384, 
        #     nhead=4, 
        #     batch_first=True,
        #     dropout=dropout, 
        #     device=device,
        # )
        self.transformer = Transformer(
            src_vocab_size=NUM_VERBS,
            trg_vocab_size=NUM_VERBS,
            src_pad_idx=None,
            embed_size=384,
            dropout=0.1,
            max_length=6,
            device=device,
        )

        self.fc_out = nn.Linear(100*WORD_EMBEDDING_SIZE, NUM_VERBS)
        
    def _get_target_mask(self, feats):
        trg_len = feats.shape[1]
        trg_mask = torch.tril(torch.ones(trg_len, trg_len))
        return trg_mask.to(self.device)
    
    def forward(self, x):
        N = 6 #! do not hardcode
        feats = self.feature_model(x, permute_dims=False)
        feats = self.transformer(feats, feats)
        # target_mask = self._get_target_mask(feats)
        # feats = self.transformer(feats, feats, tgt_mask=target_mask).reshape((N, -1))
        # print(f'feats shape = {feats.shape}')
        return self.fc_out(feats)

In [5]:
from torch.optim import AdamW

device = torch.device('cuda:0')
haarModel = HaarModel(system.verb_map, system.noun_map, 0.1, device)
training_dataset = data_module.train_dataloader()
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = AdamW(haarModel.parameters(), lr=cfg.learning.lr, weight_decay=5e-4)

/coc/pcba1/bdas31/Miniconda3/envs/haar/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/coc/pcba1/bdas31/Miniconda3/envs/haar/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


2023-08-09 18:10:12,450 [MainThread  ] [WARNI]  model.pretrained was set to 'None' but you also specified to load weights from {'rgb': '/coc/scratch/bdas31/EPIC-Kitchens_100/C1-Action-Recognition-TSN-TRN-TSM/pretrained_models/tsm_rgb.ckpt', 'flow': '/coc/scratch/bdas31/EPIC-Kitchens_100/C1-Action-Recognition-TSN-TRN-TSM/pretrained_models/tsm_flow.ckpt'}.The latter will take precedence.
2023-08-09 18:10:12,948 [MainThread  ] [WARNI]  model.pretrained was set to 'None' but you also specified to load weights from {'rgb': '/coc/scratch/bdas31/EPIC-Kitchens_100/C1-Action-Recognition-TSN-TRN-TSM/pretrained_models/tsm_rgb.ckpt', 'flow': '/coc/scratch/bdas31/EPIC-Kitchens_100/C1-Action-Recognition-TSN-TRN-TSM/pretrained_models/tsm_flow.ckpt'}.The latter will take precedence.


In [8]:
print(sum(p.numel() for p in haarModel.parameters() if p.requires_grad))

80855954


In [6]:
from sklearn.metrics import accuracy_score
import warnings

haarModel.to(device)
haarModel.train()
warnings.filterwarnings('ignore')
step_loss = step_acc = 0.
for i, batch in enumerate(training_dataset):
    labels = batch[0][1]['verb_class']
    preds = haarModel(batch).cpu()
    loss = loss_fn(preds, labels)
    with torch.no_grad():
        y = torch.argmax(nn.Softmax(dim=1)(preds), dim=-1)
        accuracy = accuracy_score(labels, y)
        step_loss += loss.item()
        step_acc += accuracy

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if i == 0:
        print(f'First step - Loss: {step_loss:.4f} Accuracy: {step_acc:.4f}')

    if (i + 1) % 10 == 0:
        avg_loss = step_loss / 10
        avg_acc = step_acc / 10
        print(f'Step {i + 1} - Loss: {avg_loss:.4f} Accuracy: {avg_acc:.4f}')
        step_loss = step_acc = 0.
    

trg shape = torch.Size([6, 100, 384])


ValueError: too many values to unpack (expected 2)

In [32]:
from train import Trainer
# print(device)
trainer = Trainer(name='transformers_testing', cfg=cfg, device='cpu')
# trainer.train(2)


transformers_testing_../../data/pilot-02/logs/train_temp_transformer.log
transformers_testing_../../data/pilot-02/logs/train_temp_transformer.log


In [34]:
trainer.train(2)

  0%|          | 0/2 [00:00<?, ?it/s]/coc/pcba1/bdas31/Miniconda3/envs/haar/lib/python3.8/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


First step - Loss: 4.5295 Accuracy: 0.0000


/coc/pcba1/bdas31/Miniconda3/envs/haar/lib/python3.8/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():
/coc/pcba1/bdas31/Miniconda3/envs/haar/lib/python3.8/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():
/coc/pcba1/bdas31/Miniconda3/envs/haar/lib/python3.8/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():
/coc/pcba1/bdas31/Miniconda3/envs/haar/lib/python3.8/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.

Step 10 - Loss: 3.8743 Accuracy: 0.0500


/coc/pcba1/bdas31/Miniconda3/envs/haar/lib/python3.8/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():
/coc/pcba1/bdas31/Miniconda3/envs/haar/lib/python3.8/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():
/coc/pcba1/bdas31/Miniconda3/envs/haar/lib/python3.8/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():
/coc/pcba1/bdas31/Miniconda3/envs/haar/lib/python3.8/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.

...

Step 50 - Loss: 3.7703 Accuracy: 0.3000


/coc/pcba1/bdas31/Miniconda3/envs/haar/lib/python3.8/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():
/coc/pcba1/bdas31/Miniconda3/envs/haar/lib/python3.8/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():
/coc/pcba1/bdas31/Miniconda3/envs/haar/lib/python3.8/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():
/coc/pcba1/bdas31/Miniconda3/envs/haar/lib/python3.8/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.

Step 60 - Loss: 3.5360 Accuracy: 0.1250


/coc/pcba1/bdas31/Miniconda3/envs/haar/lib/python3.8/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():
61it [24:19, 23.92s/it]
  0%|          | 0/2 [24:19<?, ?it/s]


KeyboardInterrupt: 

In [49]:
import warnings
from tqdm import tqdm
from time import sleep
a = [(i, i*2 + 1) for i in range(5)]
for i, num in tqdm(enumerate(a), total=len(a), desc='epochs'):
    sleep(1)
    print('new epoch')
    for j in tqdm(range(5), desc='inner loop', position=0, leave=False):
        sleep(0.2)



epochs:   0%|          | 0/5 [00:00<?, ?it/s]

new epoch


epochs:  20%|██        | 1/5 [00:02<00:08,  2.01s/it]    

new epoch


epochs:  40%|████      | 2/5 [00:04<00:06,  2.01s/it]    

...

epochs:  60%|██████    | 3/5 [00:06<00:04,  2.01s/it]    

new epoch


epochs:  80%|████████  | 4/5 [00:08<00:02,  2.01s/it]    

new epoch


epochs: 100%|██████████| 5/5 [00:10<00:00,  2.01s/it]    


In [41]:
trnsf = nn.Transformer(d_model=100, nhead=4, batch_first=True)
attention = trnsf(feats, feats)
print(attention.shape)

torch.Size([6, 384, 100])


In [42]:
print(attention.grad_fn)

In [17]:
import pprint as pp

# pp = pprint()
training_dataset = data_module.train_dataloader()
eval_dataset = data_module.val_dataloader()
# print(iter(training_dataset).device)

def train_ddp_huggingface():
    training_args = TrainingArguments(
        output_dir=cfg.model.save_path,
        per_device_train_batch_size= cfg.learning.batch_size,
        per_device_eval_batch_size=cfg.learning.val_batch_size,
        num_train_epochs=5,
        evaluation_strategy='epoch',
        remove_unused_columns=False,
        log_level="debug",
        log_on_each_node=False,
        logging_dir=cfg.trainer.tb_runs,
        logging_steps=cfg.trainer.log_every_n_steps
    )


    def collate_fn(examples):
        '''examples = [(R, F) * bs]
        '''

        # print(len(examples), len(examples[0]), examples[0][0][0].shape)
        rgb_images = []
        flow_images = []
        labels = []
        narration = []
        for eg in examples:
            rgb_images.append(eg[0][0])
            flow_images.append(eg[1][0])
            labels.append(eg[0][1]['verb_class'])
            narration.append(eg[0][1]['narration'])
            # pp.pprint(dct)
            # print(eg[0][1])
            # print(len(eg[0][1]))
            # print(eg[0][0].shape)
            # break
        # print(len(rgb_images))
        # rgb, flow = examples
        # rgb_images, metadata = rgb  
        # flow_images = flow[0]
        # labels = metadata["verb_class"]
        # text = metadata["narration"]
        return {
            "rgb_images": torch.stack(rgb_images),
            "flow_images": torch.stack(flow_images),
            "narration": narration,
            "labels": labels,
        }


    class MyTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False):
            # rgb_images = inputs['rgb_images']
            # # print(type(rgb_images), len(rgb_images), type(rgb_images[0]))
            # rgb_images = torch.tensor([rgb_images])
            print(next(system.rgb_model.parameters()).device, inputs['rgb_images'].device)
            rgb_feats = system.rgb_model(inputs['rgb_images'])
            flow_feats = system.flow_model(inputs['flow_images'])
            narration_feats = system.narration_model(input['narration'])
            feats = torch.hstack([rgb_feats, flow_feats, narration_feats])

            preds = model(feats, inputs['labels'], system.verb_embeddings)
            targets=inputs["labels"]
            return self.compute_loss(preds, targets), preds if return_outputs else \
                self.compute_loss(preds, targets)
        
    trainer = MyTrainer(
        model,
        training_args,
        train_dataset=training_dataset.dataset,
        eval_dataset=eval_dataset.dataset,
        data_collator=collate_fn,
    )

    trainer.train()

In [20]:
for batch in training_dataset:
    # print(len(batch))
    print(batch[0][1])
    

OrderedDict([('index', tensor([ 3131, 28357, 18080,  6579, 22033, 17588])), ('Unnamed: 0', tensor([ 3131, 28358, 18080,  6579, 22033, 17588])), ('video_id', ['P02_113', 'P37_101', 'P22_117', 'P03_112', 'P27_103', 'P22_116']), ('root_dir', ['../2g1n6qdydwa9u22shpxqzp0t8m/P02', '../2g1n6qdydwa9u22shpxqzp0t8m/P37', '../2g1n6qdydwa9u22shpxqzp0t8m/P22', '../2g1n6qdydwa9u22shpxqzp0t8m/P03', '../2g1n6qdydwa9u22shpxqzp0t8m/P27', '../2g1n6qdydwa9u22shpxqzp0t8m/P22']), ('narration_id', ['P02_113_106', 'P37_101_216', 'P22_117_405', 'P03_112_65', 'P27_103_306', 'P22_116_136']), ('narration', ['clear chopping board', 'smash garlic', 'remove lid', 'pick up bowl', 'stir liquid', 'pick up spoon']), ('narration_timestamp', ['00:04:47.905', '00:15:02.908', '00:14:56.851', '00:02:05.851', '00:21:54.101', '00:05:19.861']), ('start_frame', tensor([14296, 45046, 44743,  6193, 65606, 15894])), ('end_frame', tensor([14483, 45408, 44918,  6381, 65756, 16044])), ('verb_class', tensor([12, 49, 12,  0, 10,  0])),

KeyboardInterrupt: 

In [ ]:
from accelerate import notebook_launcher
# os.environ['TOKENIZERS_PARALLELISM']="false"
notebook_launcher(train_ddp_huggingface, args=(), num_processes=1)